In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('/home/ecapi/code/e-capi/Chat_bot_Software_Engineer/Chat_bot_Software_Engineer/data/simulation-db')

In [3]:
c = conn.cursor()

In [5]:
c.execute("""SELECT *
          From sales
          Where id_sale Like '123456'""")
rows = c.fetchall()
rows

[(123456,
  384923,
  'CHIP37648',
  '2021-12-01 12:00:00-03:00',
  'in progress',
  'in the delibery step')]